In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow import keras
import river
from river.stream import iter_pandas
from river.metrics import Accuracy,BalancedAccuracy,CohenKappa,GeometricMean,ROCAUC,F1
from river.evaluate import progressive_val_score
from river.naive_bayes import GaussianNB
from river.neighbors import KNNClassifier
from river.forest import ARFClassifier
from river.tree import HoeffdingTreeClassifier
from river.tree import HoeffdingAdaptiveTreeClassifier

2024-03-18 13:18:00.853640: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from river_torch import classification
from torch import nn
from torch import optim
from torch import manual_seed

In [4]:
# FAN PROJECT
test_data = np.load('/Users/saratonazzi/Documents/Tesi/Project/project_data/EdgeImpulse/fan/ei-industry-4.0---predictivemaintenance---fan-spectrogram-X_testing.npy')
train_data = np.load('/Users/saratonazzi/Documents/Tesi/Project/project_data/EdgeImpulse/fan/ei-industry-4.0---predictivemaintenance---fan-spectrogram-X_training.npy')
train_labels = np.load('/Users/saratonazzi/Documents/Tesi/Project/project_data/EdgeImpulse/fan/ei-industry-4.0---predictivemaintenance---fan-spectrogram-y_training.npy')
test_labels = np.load('/Users/saratonazzi/Documents/Tesi/Project/project_data/EdgeImpulse/fan/ei-industry-4.0---predictivemaintenance---fan-spectrogram-y_testing.npy')

In [22]:
# MY FAN2
test_data = np.load('/Users/saratonazzi/Documents/Python/Tesi/fan2/ei-industry-4.0---predictivemaintenance---fan-spectrogram-X_testing.npy')
train_data = np.load('/Users/saratonazzi/Documents/Python/Tesi/fan2/ei-industry-4.0---predictivemaintenance---fan-spectrogram-X_training.npy')
train_labels = np.load('/Users/saratonazzi/Documents/Python/Tesi/fan2/ei-industry-4.0---predictivemaintenance---fan-spectrogram-y_training.npy')
test_labels = np.load('/Users/saratonazzi/Documents/Python/Tesi/fan2/ei-industry-4.0---predictivemaintenance---fan-spectrogram-y_testing.npy')

In [62]:
# MY FAN
test_data = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/fan/ei-industry-4.0---predictivemaintenance---fan-spectrogram-X_testing.npy')
train_data = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/fan/ei-industry-4.0---predictivemaintenance---fan-spectrogram-X_training.npy')
train_labels = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/fan/ei-industry-4.0---predictivemaintenance---fan-spectrogram-y_training.npy')
test_labels = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/fan/ei-industry-4.0---predictivemaintenance---fan-spectrogram-y_testing.npy')

In [6]:
# MY PUMP
test_data = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/pump/ei-industry-4.0---predictive-maintenance---pump-mfe-X_testing.npy')
train_data = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/pump/ei-industry-4.0---predictive-maintenance---pump-mfe-X_training.npy')
train_labels = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/pump/ei-industry-4.0---predictive-maintenance---pump-mfe-y_training.npy')
test_labels = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/pump/ei-industry-4.0---predictive-maintenance---pump-mfe-y_testing.npy')

In [172]:
# MY VALVE
test_data = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/valve/ei-industry-4.0---predictive-maintenance---valve-spectrogram-X_testing.npy')
train_data = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/valve/ei-industry-4.0---predictive-maintenance---valve-spectrogram-X_training.npy')
train_labels = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/valve/ei-industry-4.0---predictive-maintenance---valve-spectrogram-y_training.npy')
test_labels = np.load('/Users/saratonazzi/Documents/Python/Tesi/mydata/valve/ei-industry-4.0---predictive-maintenance---valve-spectrogram-y_testing.npy')

In [7]:
train_labels[train_labels[:,0]==2,0] = 0
test_labels[test_labels[:,0]==2,0] = 0

In [10]:
model1 = keras.models.load_model('Documents/Python/Tesi/mydata/pump/model.h5')
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 299, 128)          0         
                                                                 
 conv1d (Conv1D)             (None, 299, 8)            3080      
                                                                 
 max_pooling1d (MaxPooling1  (None, 150, 8)            0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 150, 8)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 150, 16)           400       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 75, 16)            0         
 g1D)                                                   

In [65]:
#model.pop()
#model.summary()

In [11]:
feature_extractor = keras.Model(
   inputs=model1.inputs,
   outputs=model1.get_layer(name="flatten").output)

#feature_extractor = keras.Model(
#   inputs=model.inputs,
#   outputs=model.get_layer(name="dense").output)

features = feature_extractor.predict(train_data)

#print(features)
print(features.shape)

58/58 [==============================] - 1s 7ms/step
(1856, 1200)


In [12]:
metrics = BalancedAccuracy()
#Accuracy()
#F1()
#ROCAUC()
#metrics = [Accuracy(),BalancedAccuracy(),ROCAUC()]
model = GaussianNB() #KNNClassifier(n_neighbors=2) #HoeffdingTreeClassifier() #ARFClassifier(seed=42) #KNNClassifier(n_neighbors=2) #HoeffdingAdaptiveTreeClassifier(seed=42)
#model = KNNClassifier(n_neighbors=5, window_size=1000)
model.window = 1000

features_df = pd.DataFrame(features)
#labels_df = pd.DataFrame(train_labels[:,0])
labels_series = pd.Series(train_labels[:,0])
stream = iter_pandas(X=features_df, y=labels_series)

In [74]:
#shuffle the order of data grouped by 9

distinct = len(set(train_labels[:,1]))

features_shuffle = np.zeros(features.shape)
train_labels_shuffle = np.zeros(train_labels[:,0].shape)

positions = random.sample(range(distinct),distinct)

for i in range(distinct):
    pos = positions[i]
    features_shuffle[i*9:i*9+8,:] = features[pos*9:pos*9+8,:]
    train_labels_shuffle[i*9:i*9+8] = train_labels[pos*9:pos*9+8,0]

ValueError: could not broadcast input array from shape (0,1200) into shape (8,1200)

In [75]:
#shuffle the order of data

samples = features.shape[0]

features_shuffle = np.zeros(features.shape)
train_labels_shuffle = np.zeros(train_labels[:,0].shape)

positions = random.sample(range(samples),samples)

for i in range(samples):
    pos = positions[i]
    features_shuffle[i,:] = features[pos,:]
    train_labels_shuffle[i] = train_labels[pos,0]

In [76]:
features_df_shuffle = pd.DataFrame(features_shuffle)
labels_series_shuffle = pd.Series(train_labels_shuffle)
stream_shuffle = iter_pandas(X=features_df_shuffle, y=labels_series_shuffle)
#model = ARFClassifier() #HoeffdingAdaptiveTreeClassifier() #GaussianNB() #KNNClassifier(n_neighbors=2, window_size=1000) #HoeffdingTreeClassifier() #HoeffdingAdaptiveTreeClassifier(seed=42)
#metrics = BalancedAccuracy()

In [84]:
progressive_val_score(dataset=stream_shuffle,
                      model=model,
                      metric=metrics,
                      print_every=100)

NameError: name 'stream_shuffle' is not defined

In [165]:
progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=100)

[100] BalancedAccuracy: 100.00%
[200] BalancedAccuracy: 98.68%
[300] BalancedAccuracy: 98.91%
[400] BalancedAccuracy: 98.69%
[500] BalancedAccuracy: 98.69%
[600] BalancedAccuracy: 98.69%
[700] BalancedAccuracy: 98.69%
[800] BalancedAccuracy: 98.69%
[900] BalancedAccuracy: 98.69%
[1,000] BalancedAccuracy: 98.69%
[1,100] BalancedAccuracy: 98.69%
[1,200] BalancedAccuracy: 97.83%
[1,300] BalancedAccuracy: 93.73%
[1,400] BalancedAccuracy: 94.75%
[1,500] BalancedAccuracy: 95.22%
[1,600] BalancedAccuracy: 96.06%
[1,700] BalancedAccuracy: 96.42%
[1,800] BalancedAccuracy: 96.88%
[1,900] BalancedAccuracy: 97.23%
[2,000] BalancedAccuracy: 97.46%
[2,100] BalancedAccuracy: 97.70%
[2,200] BalancedAccuracy: 97.89%
[2,300] BalancedAccuracy: 98.05%
[2,400] BalancedAccuracy: 98.19%
[2,500] BalancedAccuracy: 98.31%
[2,600] BalancedAccuracy: 98.42%
[2,700] BalancedAccuracy: 98.51%
[2,800] BalancedAccuracy: 98.59%
[2,900] BalancedAccuracy: 98.66%
[3,000] BalancedAccuracy: 98.73%
[3,087] BalancedAccuracy: 9

BalancedAccuracy: 98.78%

In [166]:
features2 = feature_extractor.predict(test_data)
features_df2 = pd.DataFrame(features2)
labels_series2 = pd.Series(test_labels[:,0])
stream2 = iter_pandas(X=features_df2, y=labels_series2)
metrics= BalancedAccuracy()
#ROCAUC()
#Accuracy()
#BalancedAccuracy()
#F1()

24/24 [==============================] - 1s 14ms/step


In [167]:
progressive_val_score(dataset=stream2,
                      model=model,
                      metric=metrics,
                      delay=10,
                      print_every=100)

[100] BalancedAccuracy: 88.89%
[200] BalancedAccuracy: 96.35%
[300] BalancedAccuracy: 97.05%
[400] BalancedAccuracy: 97.84%
[500] BalancedAccuracy: 97.56%
[600] BalancedAccuracy: 95.30%
[700] BalancedAccuracy: 95.71%
[756] BalancedAccuracy: 95.99%


BalancedAccuracy: 95.99%